# Multiple fit code

In [57]:
import os
import scipy
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from math import factorial
from scipy import optimize
from matplotlib.pyplot import cm
from lmfit import Model
from matplotlib.gridspec import GridSpec
from scipy.stats import poisson as pois
from scipy.stats import norm

import nbimporter
import fitFunc as fits

from symfit import variables, parameters, Fit
import symfit as sm
import numpy as np

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit

## Multiple loading (load + prep)

In [118]:
def multipleLoad(path='db/',bin_width=651,nbins=100):
    
    def list_files(path):
        listFile=[]
        for root, dirs, files in os.walk(path):
            for file in files:
                listFile.append(os.path.join(root, file))
        return listFile
                
    
    file_list=list_files(path)
    InfoDataset=[]
    
    for file in file_list:
        
        meta = pd.read_excel(file,sheet_name=0,header=None)
        freq = pd.read_excel(file,sheet_name=1)             # frequecies
        fft  = pd.read_excel(file,sheet_name=2)             # power
    
        data = pd.DataFrame({'freq':freq[1]})
    
        col = 0
        for col_name in fft.columns: # load all the subruns
            if col > 0:
                data[f'fft{col-1}'] = fft[col_name]
            col += 1
        #cavity frequency and number of files per run
        center = meta[1][3]
        length = meta[1][8]
        N = length*2731 #N=1365500 if length=500
        
        mask = (data['freq']>center-bin_width*nbins) & (data['freq']<center+bin_width*nbins)
        cavdata = data[mask].reset_index(drop=True)
        
        cavdata["fft"] = cavdata.iloc[:,1:].mean(axis=1)
        cavdata=cavdata[["freq","fft"]]
        
        minW = np.min(cavdata["fft"].copy()) # minimum power in the cavity
        
        # In general, the average measured power should be known and equal to the noise temperature of the system.
        # So we can rescale the data so that the power at the cavity frequency sia T_noise k_b B (W)
        ref = minW**(-1) * 3.5*1.38e-23*651/1e-24 
        
        cavdata["fft"] = ref * cavdata["fft"]   #y' 
        
        # set weights -> sqrt(sigma'/N)=ref*sqrt(y/N)
        weights = np.sqrt(ref)*np.sqrt(cavdata["fft"])/np.sqrt(N)         
        
        Info={"name":file,"length":length,"center":center,"cavdata":cavdata,"weights":weights,"ref":ref} 
        InfoDataset.append(Info)
            
    return(InfoDataset)

## Multiple fit

In [124]:
def multipleFitBKG(InfoDataset):
    
    fitResult=[]
    for run in InfoDataset:
        bkg_result=fits.fit_bkg(run["cavdata"]["freq"],run["cavdata"]["fft"],
                                run['weights'],run["center"],run["ref"])
            
        results={"run":run["name"],
                 "background_bestParams":bkg_result.params.valuesdict(),
                 "background_residuals":bkg_result.residual,
                 "background_bestfit":bkg_result.best_fit}
            
        fitResult.append(results)
    
    return(fitResult)

In [133]:
def multipleFitSIG(InfoDataset,fitBkg,x0_,mu_=1,sigma_=16*651):
    
    #variables
    xs=variables(', '.join('x_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    ys=variables(', '.join('y_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    
    #specific parameters
    ap=parameters(', '.join('a_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    bp=parameters(', '.join('b_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    cp=parameters(', '.join('c_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    dp=parameters(', '.join('d_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    ep=parameters(', '.join('e_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    fp=parameters(', '.join('f_{}'.format(i) for i in range(1, len(fitBkg)+1)))
    
    for i in range(len(fitBkg)):
        ap[i].fixed=fits_bkg[i]["background_bestParams"]["a"]
        bp[i].fixed=fits_bkg[i]["background_bestParams"]["b"]
        cp[i].fixed=fits_bkg[i]["background_bestParams"]["c"]
        dp[i].fixed=fits_bkg[i]["background_bestParams"]["d"]
        ep[i].fixed=fits_bkg[i]["background_bestParams"]["e"]
        fp[i].fixed=fits_bkg[i]["background_bestParams"]["f"]
    
    #common parameters
    x0,mu,sigma=parameters("x0,mu,sigma")
    
    x0.fixed=x0_
    sigma.fixed=sigma_
    mu.value=mu_
    
    model_dict = {
            y: e**2*sm.Abs(x-a+1*sm.I*b)**2/sm.Abs(x-c+1*sm.I*d)**2+f*(x-c) 
                + mu*(1/(sm.sqrt(2*sm.pi)*sigma)) * sm.exp(-.5*((x-x0)/sigma)**2)
            
            for x, y,a,b,c,d,e,f in zip(xs, ys, ap,bp,cp,dp,ep,fp)
        }
    
    x=np.empty((len(fitBkg),200))
    y=np.empty((len(fitBkg),200))
    
    i=0
    for run in InfoDataset:
        x[i]=run["cavdata"]["freq"]
        y[i]=run["cavdata"]["fft"]
        i+=1
            
    fit = Fit(model_dict, 
                    x_1=x[0], y_1=y[0],
                    x_2=x[1], y_2=y[1],
                    x_3=x[2], y_3=y[2],
                    x_4=x[3], y_4=y[3],
                    x_5=x[4], y_5=y[4],
                    x_6=x[5], y_6=y[5],
                    x_7=x[6], y_7=y[6],
                    x_8=x[7], y_8=y[7],
                    x_9=x[8], y_9=y[8],
                    x_10=x[9], y_10=y[9],
                    x_11=x[10], y_11=y[10],
                    x_12=x[11], y_12=y[11],
                    x_13=x[12], y_13=y[12])
    
    fit_result = fit.execute()
    
    #results
    mu_hat=fit_result.value(mu)

    yfit=np.empty((len(fitBkg),200))
    for i in range(len(fitBkg)):
        yfit[i]=fits.signal_gauss(x[i],
                                  fit_result.value(ap[i]),
                                  fit_result.value(bp[i]),
                                  fit_result.value(cp[i]),
                                  fit_result.value(dp[i]),
                                  fit_result.value(ep[i]),
                                  fit_result.value(fp[i]),
                                  fit_result.value(x0),
                                  fit_result.value(sigma),
                                  fit_result.value(mu))
                                  
    i=0
    resultsFit={}
    for run in InfoDataset:
        bestFit=pd.DataFrame({"freq":x[i],"fft":y[i]})
        fitsRes={"run":run["name"],"bestFits":bestFit}
        resultsFit.append(fitsRes)
        i+=1
    
    
    return(resultsFit,mu_hat)

## CI

In [ ]:
def CI_mu(dataRuns,
          x_0=np.array([]), mu_fix=np.array([]), n_toy=500,
          signal=fits.signal_gauss):
    
    #n_toy : number of toypoints for each x0 for each func(background and signal)
    #x_0 : x0 to be tested
    #mu_fix : mu to be tested
    #signal : which signal
    
    # fit background once
    res_bkg = multipleFitBKG(dataRuns)
    
    bkg        = {}
    bkg_params = {}
    toy_0 = {}
    for fit in res_bkg:
        
        bestFit={"run":fit['run'],"bestfit":fit["background_bestfit"]}
        bestParams={"run":fit['run'],"bestParams":fit["background_bestParams"]}
        bkg.append(bestFit)
        bkg.append(bestParams)
        
        toy_bkg = stats.gen_toydataset(values=fit["background_bestfit"], n=n_toy)
        toy_0.append({"run":fit['run'],"toy_0":toy_bkg})
    
    mu_CI = np.empty(len(x_0))
    
    for i_x0 in range(len(x_0)):
        mu_CI[i_x0] = CI() #we need to implement this function
    return mu_CI

## TEST

### Fit

In [121]:
runsData=multipleLoad()

{'name': 'db/AnalyzedDataFFT_Run_397_sliced.xlsx',
 'length': 500.0,
 'center': 10353439835.0,
 'cavdata':              freq           fft
 0    1.035338e+10  34980.892474
 1    1.035338e+10  35071.619056
 2    1.035338e+10  34994.358260
 3    1.035338e+10  34987.893835
 4    1.035338e+10  35049.954986
 ..            ...           ...
 195  1.035350e+10  34357.663612
 196  1.035350e+10  34372.673747
 197  1.035350e+10  34383.512465
 198  1.035350e+10  34326.823812
 199  1.035350e+10  34400.354812
 
 [200 rows x 2 columns],
 'weights': 0      4223.045975
 1      4228.518883
 2      4223.858721
 3      4223.468571
 4      4227.212683
           ...     
 195    4185.257452
 196    4186.171577
 197    4186.831537
 198    4183.378665
 199    4187.856846
 Name: fft, Length: 200, dtype: float64,
 'ref': 696165405.1618804}

In [130]:
fitBkg=multipleFitBKG(runsData)
fits_sig=multipleFitSIG(runsData,fitBkg,10353357599.0)

13

### CI